In [1]:
!pip install datasets transformers accelerate torch scikit-learn matplotlib seaborn

  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\LOQ\\AppData\\Local\\Temp\\pip-unpack-p2foji14\\fsspec-2025.3.0-py3-none-any.whl.metadata'
Consider using the `--user` option or check the permissions.



In [3]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('jbarat/plant_species')

dataset

ModuleNotFoundError: No module named 'datasets'

In [ ]:
dataset['train'].features

In [ ]:
names=['aechmea_fasciata', 
'agave_americana', 
'agave_attenuata', 
'agave_tequilana', 
'aglaonema_commutatum', 
'albuca_spiralis',
 'allium_cepa', 
'allium_sativum']

In [ ]:
# Split the training data into train and test (let's say 10% for the test set)
train_test_split = dataset['train'].train_test_split(test_size=0.1)

# Further split the training set to get a validation set (e.g., 10% of the training set)
train_val_split = train_test_split['train'].train_test_split(test_size=0.1)

# Combine the splits into a new DatasetDict
final_dataset = {
    'train': train_val_split['train'],
    'val': train_val_split['test'],  
    'test': train_test_split['test']  
}

final_dataset

In [ ]:
train_ds = final_dataset["train"]
val_ds = final_dataset["val"]
test_ds = final_dataset["test"]

In [ ]:
image = train_ds[1]['image']
image

In [ ]:
import matplotlib.pyplot as plt

shown_labels = set()

plt.figure(figsize=(15, 10))

# Loop through the dataset and plot the first image of each label
for i, sample in enumerate(train_ds):
    label = train_ds.features["label"].names[sample["label"]]
    if label not in shown_labels:
        plt.subplot(1, len(train_ds.features["label"].names), len(shown_labels) + 1)
        plt.imshow(sample["image"])
        plt.title(label)
        plt.axis("off")
        shown_labels.add(label)
        if len(shown_labels) == len(train_ds.features["label"].names):
            break

plt.show()

In [ ]:
id2label = {id: label for id, label in enumerate(train_ds.features["label"].names)}
label2id = {label: id for id, label in id2label.items()}
id2label, id2label[train_ds[0]["label"]]

In [ ]:
from transformers import ViTImageProcessor

model_name = "google/vit-large-patch16-224"
processor = ViTImageProcessor.from_pretrained(model_name)
processor

In [ ]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    ToTensor,
    Resize,
)

# Get configurations from ViT processor
image_mean, image_std = processor.image_mean, processor.image_std
size = processor.size["height"]

# Normalizes the image pixels by subtracting the mean and dividing by the std from the pretrained model configurations
normalize = Normalize(mean=image_mean, std=image_std)

# Compose: Combines a series of image transformations into one pipeline.
train_transforms = Compose(
    [
        RandomResizedCrop(size),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize,
    ]
)
val_transforms = Compose(
    [
        Resize(size),
        CenterCrop(size),
        ToTensor(),
        normalize,
    ]
)
test_transforms = Compose(
    [
        Resize(size),
        CenterCrop(size),
        ToTensor(),
        normalize,
    ]
)

In [ ]:
def apply_train_transforms(examples):
    examples["pixel_values"] = [train_transforms(image.convert("RGB")) for image in examples["image"]]
    return examples


def apply_val_transforms(examples):
    examples["pixel_values"] = [val_transforms(image.convert("RGB")) for image in examples["image"]]
    return examples


def apply_test_transforms(examples):
    examples["pixel_values"] = [val_transforms(image.convert("RGB")) for image in examples["image"]]
    return examples

train_ds.set_transform(apply_train_transforms)
val_ds.set_transform(apply_val_transforms)
test_ds.set_transform(apply_test_transforms)

In [ ]:
import torch
from torch.utils.data import DataLoader


def collate_fn(examples):
    # Stacks the pixel values of all examples into a single tensor and collects labels into a tensor
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

# Create a DataLoader for the training dataset, with custom collation and a batch size of 4
train_dl = DataLoader(train_ds, collate_fn=collate_fn, batch_size=4)

In [ ]:
batch = next(iter(train_dl))
for k, v in batch.items():
    if isinstance(v, torch.Tensor):
        print(k, v.shape)

# Output
# pixel_values torch.Size([4, 3, 224, 224])
# labels torch.Size([4])

In [ ]:
from transformers import ViTForImageClassification

labels  = dataset['train'].features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name, 
    num_labels = len(labels),
    id2label=id2label, 
    label2id=label2id, 
    ignore_mismatched_sizes=True
)

In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np

train_args = TrainingArguments(
    output_dir="output-models",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=2,
  fp16=True,
  save_steps=10,
  eval_steps=10,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model,
    train_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    tokenizer=processor,
)
trainer.train()

In [ ]:
outputs = trainer.predict(test_ds)
print(outputs.metrics)

# Output
# {'test_loss': 0.25027137994766235, 
# 'test_runtime': 1.3596, 
# 'test_samples_per_second': 58.842, 
# 'test_steps_per_second': 7.355}

In [ ]:
from sklearn.metrics import classification_report

target_names = id2label.values()

print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_true = outputs.label_ids
y_pred = outputs.predictions.argmax(1)

labels = train_ds.features["label"].names
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(xticks_rotation=45)

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import timm

# Load model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.jit.load('vit_jit_emotion_model.pt').to(device)
model.eval()

# Class labels (update with your emotions)
class_names = ['angry', 'boring', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'stress', 'suprised']

# Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def predict_emotion(image_path):
    img = Image.open(image_path).convert('RGB')
    input_tensor = transform(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        confidence, class_idx = torch.max(probabilities, 0)
    
    return class_names[class_idx.item()], confidence.item()

# Example usage
emotion, confidence = predict_emotion('test_image.jpg')
print(f'Predicted Emotion: {emotion} | Confidence: {confidence:.2%}')

In [ ]:
import cv2
import torch
from torchvision import transforms
from PIL import Image

# Load transformer model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.jit.load('D:/RuhunaNew/Academic/Research/Facial_Recog_Repo/Group_50_Repo/Final_Best_Codes/Models/vit_jit_emotion_model.pt').to(device)
model.eval()

# Emotion classes
class_names = ['angry', 'boring', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'stress', 'suprised']

# Transform for face image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Haar Cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Webcam input
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Extract and preprocess the face
        face_img = frame[y:y+h, x:x+w]
        face_pil = Image.fromarray(cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB))
        input_tensor = transform(face_pil).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            probabilities = torch.nn.functional.softmax(output[0], dim=0)
            confidence, class_idx = torch.max(probabilities, 0)
            emotion = class_names[class_idx.item()]
            confidence_pct = confidence.item() * 100

        # Draw bounding box and emotion label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        label = f'{emotion} ({confidence_pct:.1f}%)'
        cv2.putText(frame, label, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow("Emotion Detection (ViT + Haar)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import timm
from timm.data import Mixup
from timm.loss import LabelSmoothingCrossEntropy
from timm.optim import AdamP
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

"""
CONFIGURATION SECTION
- Set all hyperparameters and model selection in one place
- Easily adjustable for experimentation
"""
MODEL_NAME = 'deit_small_distilled_patch16_224'  # Distilled ViT for better efficiency
NUM_CLASSES = 9                                  # Number of emotion classes
BATCH_SIZE = 64                                  # Larger batch size for stability
IMG_SIZE = 224                                   # Input image size (ViT standard)
EPOCHS = 100                                     # Maximum training epochs
LR = 5e-5                                        # Learning rate
WARMUP_EPOCHS = 5                                # Warmup period
PATIENCE = 7                                     # Early stopping patience
MIN_DELTA = 0.001                                # Minimum improvement to reset patience
MODEL_SAVE_PATH = 'best_emotion_model.pth'       # Model save location

"""
DATA AUGMENTATION PIPELINE
- Critical for improving model generalization
- Techniques applied:
  1. Random cropping (with padding)
  2. Horizontal flipping (mirror images)
  3. Affine transformations (rotation, scaling, translation)
  4. Color jitter (lighting variations)
  5. Gaussian blur (focus on structural features)
  6. Random erasing (simulate occlusions)
"""
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE + 32, IMG_SIZE + 32)),  # Slightly larger for random crop
    transforms.RandomCrop(IMG_SIZE),                    # Random crop to final size
    transforms.RandomHorizontalFlip(p=0.5),             # Mirror images
    transforms.RandomAffine(                             # Perspective/rotation variations
        degrees=15, 
        translate=(0.1, 0.1), 
        scale=(0.9, 1.1)
    ),
    transforms.ColorJitter(                             # Lighting variations
        brightness=0.2, 
        contrast=0.2, 
        saturation=0.2
    ),
    transforms.GaussianBlur(                             # Focus on structural features
        kernel_size=(5, 5), 
        sigma=(0.1, 2.0)
    ),
    transforms.ToTensor(),                               # Convert to tensor
    transforms.Normalize(                                # ImageNet normalization
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    ),
    transforms.RandomErasing(                            # Simulate occlusions
        p=0.5, 
        scale=(0.02, 0.1), 
        value='random'
    ),
])

# Validation transforms (no augmentation)
val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),     # Fixed size
    transforms.ToTensor(),                       # Convert to tensor
    transforms.Normalize(                        # Same normalization
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

"""
DATA LOADING FUNCTION
- Loads dataset from directory structure
- Creates train/validation splits (85/15)
- Applies appropriate transforms
- Sets up data loaders with shuffling and batching
"""
def prepare_data(data_dir='dataset'):
    # Load full dataset from directory structure
    full_dataset = datasets.ImageFolder(data_dir)
    
    # Create train/validation splits (85/15)
    train_size = int(0.85 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_data, val_data = random_split(
        full_dataset, 
        [train_size, val_size]
    )
    
    # Apply transforms to splits
    train_data.dataset.transform = train_transform
    val_data.dataset.transform = val_transform
    
    # Create data loaders
    train_loader = DataLoader(
        train_data, 
        batch_size=BATCH_SIZE, 
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_data, 
        batch_size=BATCH_SIZE*2,
        num_workers=2,
        pin_memory=True
    )
    
    return train_loader, val_loader, full_dataset.classes

"""
MODEL INITIALIZATION
- Loads pre-trained vision transformer
- Modifies final classification layer
- Applies regularization techniques
- Enables gradient checkpointing (memory optimization)
"""
def initialize_model(num_classes):
    # Create model with pre-trained weights
    model = timm.create_model(
        MODEL_NAME,
        pretrained=True,
        num_classes=num_classes,
        distilled=False,           # Disable distillation head
        drop_rate=0.05,             # Dropout for regularization
        drop_path_rate=0.1          # Stochastic depth
    )
    
    # Enable gradient checkpointing (memory saving)
    model.set_grad_checkpointing()
    
    return model

"""
TRAINING LOOP WITH EARLY STOPPING
Core components:
1. Mixed Precision Training - Faster computation, less memory
2. Mixup/Cutmix Augmentation - Regularization technique
3. Learning Rate Scheduling - Warmup + Cosine annealing
4. Early Stopping - Halts training when validation plateaus
"""
def train_model(model, train_loader, val_loader, num_epochs):
    # Setup device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Loss function with label smoothing (prevents overconfidence)
    criterion = LabelSmoothingCrossEntropy(smoothing=0.1)
    
    # Optimizer with weight decay (L2 regularization)
    optimizer = AdamP(model.parameters(), lr=LR, weight_decay=1e-4)
    
    # Learning rate scheduling
    num_steps = len(train_loader) * num_epochs
    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer, 
        T_max=num_steps - len(train_loader)*WARMUP_EPOCHS,
        eta_min=LR/100
    )
    
    # Warmup scheduler
    warmup_scheduler = optim.lr_scheduler.LambdaLR(
        optimizer,
        lambda step: min(1.0, (step + 1) / (len(train_loader) * WARMUP_EPOCHS))
    )
    
    # Mixed precision training (FP16)
    scaler = torch.cuda.amp.GradScaler(enabled=device.type=='cuda')
    
    # Early stopping initialization
    best_val_acc = 0.0
    epochs_no_improve = 0
    early_stop = False
    
    # Training history tracking
    history = {'train_loss': [], 'val_acc': [], 'lr': []}
    
    # Epoch loop
    for epoch in range(num_epochs):
        if early_stop:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break
            
        # Train phase
        model.train()
        train_loss = 0.0
        
        # Initialize Mixup each epoch
        mixup_fn = Mixup(
            mixup_alpha=0.2, 
            cutmix_alpha=1.0, 
            prob=0.5,
            label_smoothing=0.1,
            num_classes=NUM_CLASSES
        )
        
        # Batch loop
        for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader, 
                                                          desc=f'Epoch {epoch+1}/{num_epochs}')):
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Apply Mixup/Cutmix
            inputs, targets = mixup_fn(inputs, targets)
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Mixed precision forward pass
            with torch.cuda.amp.autocast(enabled=device.type=='cuda'):
                outputs = model(inputs)
                loss = criterion(outputs, targets)
            
            # Backpropagation with scaling
            scaler.scale(loss).backward()
            
            # Gradient clipping (prevent explosions)
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            # Weight update
            scaler.step(optimizer)
            scaler.update()
            
            # Track loss
            train_loss += loss.item()
            
            # Update learning rate
            if epoch < WARMUP_EPOCHS:
                warmup_scheduler.step()
            else:
                scheduler.step()
            
            # Store current LR
            current_lr = optimizer.param_groups[0]['lr']
            history['lr'].append(current_lr)
        
        # Validation phase
        model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
        
        # Calculate metrics
        avg_train_loss = train_loss / len(train_loader)
        val_acc = 100 * correct / total
        
        # Update history
        history['train_loss'].append(avg_train_loss)
        history['val_acc'].append(val_acc)
        
        print(f"Epoch {epoch+1:03d}/{num_epochs} | "
              f"Train Loss: {avg_train_loss:.4f} | "
              f"Val Acc: {val_acc:.2f}% | "
              f"LR: {current_lr:.2e}")
        
        """
        EARLY STOPPING MECHANISM
        - Monitors validation accuracy improvement
        - Saves best model checkpoint
        - Stops training if no improvement after PATIENCE epochs
        """
        # Check for improvement
        if val_acc > best_val_acc + MIN_DELTA:
            print(f"Validation accuracy improved: {best_val_acc:.2f}% -> {val_acc:.2f}%")
            best_val_acc = val_acc
            epochs_no_improve = 0
            
            # Save best model
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
                'loss': avg_train_loss,
            }, MODEL_SAVE_PATH)
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve}/{PATIENCE} epochs")
            
            # Check early stopping condition
            if epochs_no_improve >= PATIENCE:
                early_stop = True
                print(f"Early stopping triggered! Best accuracy: {best_val_acc:.2f}%")
    
    # Load best model weights
    checkpoint = torch.load(MODEL_SAVE_PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"Training complete. Best validation accuracy: {best_val_acc:.2f}%")
    return model, history

"""
VISUALIZATION FUNCTION
- Plots training history for analysis
- Helps identify overfitting/underfitting
"""
def plot_training_history(history):
    plt.figure(figsize=(15, 5))
    
    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Training Loss')
    plt.title('Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Accuracy plot
    plt.subplot(1, 2, 2)
    plt.plot(history['val_acc'], label='Validation Accuracy', color='orange')
    plt.title('Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.axhline(y=max(history['val_acc']), color='r', linestyle='--', 
                label=f'Best: {max(history["val_acc"]):.2f}%')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.show()

"""
MAIN EXECUTION FLOW
1. Prepare data
2. Initialize model
3. Train with early stopping
4. Save final model
5. Visualize results
"""
if __name__ == "__main__":
    # Prepare data
    train_loader, val_loader, class_names = prepare_data()
    print(f"Classes: {class_names}")
    
    # Initialize model
    model = initialize_model(NUM_CLASSES)
    print(f"Model: {MODEL_NAME} | Parameters: {sum(p.numel() for p in model.parameters())/1e6:.1f}M")
    
    # Train model
    trained_model, history = train_model(model, train_loader, val_loader, EPOCHS)
    
    # Save final model
    torch.save(trained_model.state_dict(), 'final_emotion_model.pth')
    print("Final model saved")
    
    # Convert to TorchScript for production
    scripted_model = torch.jit.script(trained_model)
    scripted_model.save('emotion_model_scripted.pt')
    print(f"Scripted model size: {os.path.getsize('emotion_model_scripted.pt')/1e6:.1f}MB")
    
    # Visualize training
    plot_training_history(history)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch.nn.parallel import DataParallel  # For multi-GPU support
import timm
from timm.data import Mixup
from timm.loss import LabelSmoothingCrossEntropy
from timm.optim import AdamP
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

# ======================================================================
# CUDA OPTIMIZATION SETUP
# ======================================================================
def setup_device():
    """Initialize CUDA devices and enable hardware optimizations"""
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"✅ CUDA detected: {torch.cuda.device_count()} GPU(s) available")
        
        # Enable cuDNN auto-tuner for optimal convolution algorithms
        torch.backends.cudnn.benchmark = True
        
        # Enable TF32 precision for Ampere GPUs (faster with minimal precision loss)
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        
        # Set default tensor type to CUDA for automatic device placement
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    else:
        device = torch.device('cpu')
        print("⚠️ Using CPU - performance will be limited")
    
    return device

# Initialize device at start
device = setup_device()

# ======================================================================
# CONFIGURATION (ADJUSTABLE PARAMETERS)
# ======================================================================
MODEL_NAME = 'deit_small_distilled_patch16_224'  # Distilled ViT for better efficiency
NUM_CLASSES = 9                                  # Number of emotion classes
BATCH_SIZE = 128 if torch.cuda.is_available() else 64  # Larger batches for GPUs
IMG_SIZE = 224                                   # Input image size (ViT standard)
EPOCHS = 100                                     # Maximum training epochs
LR = 5e-5                                        # Learning rate
WARMUP_EPOCHS = 5                                # Warmup period
PATIENCE = 7                                     # Early stopping patience
MIN_DELTA = 0.001                                # Minimum improvement to reset patience
MODEL_SAVE_PATH = 'best_emotion_model.pth'       # Model save location

# ======================================================================
# DATA AUGMENTATION PIPELINE
# ======================================================================
def create_transforms():
    """Create optimized data augmentation pipelines with CUDA-aware operations"""
    # Training augmentations (more aggressive)
    train_transform = transforms.Compose([
        transforms.Resize((IMG_SIZE + 32, IMG_SIZE + 32)),
        transforms.RandomCrop(IMG_SIZE),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomAffine(
            degrees=15, 
            translate=(0.1, 0.1), 
            scale=(0.9, 1.1)
        ),
        transforms.ColorJitter(
            brightness=0.2, 
            contrast=0.2, 
            saturation=0.2
        ),
        transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 2.0)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.5, scale=(0.02, 0.1), value='random'),
    ])

    # Validation transforms (minimal processing)
    val_transform = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return train_transform, val_transform

# ======================================================================
# DATA LOADING WITH CUDA OPTIMIZATION
# ======================================================================
def prepare_data(data_dir='dataset'):
    """Load and prepare dataset with CUDA-optimized data loading"""
    train_transform, val_transform = create_transforms()
    
    # Load dataset from directory structure
    full_dataset = datasets.ImageFolder(data_dir)
    
    # Split dataset (85% train, 15% validation)
    train_size = int(0.85 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_data, val_data = random_split(full_dataset, [train_size, val_size])
    
    # Apply transforms
    train_data.dataset.transform = train_transform
    val_data.dataset.transform = val_transform
    
    # Configure DataLoader for optimal GPU utilization
    num_workers = os.cpu_count()
    train_loader = DataLoader(
        train_data, 
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=min(8, num_workers),  # Optimize worker count
        pin_memory=True,      # Faster data transfer to GPU
        persistent_workers=True,  # Maintain workers between epochs
        prefetch_factor=2     # Preload batches in background
    )
    
    val_loader = DataLoader(
        val_data, 
        batch_size=BATCH_SIZE * 2,  # Larger batches for validation
        num_workers=min(4, num_workers//2),
        pin_memory=True
    )
    
    return train_loader, val_loader, full_dataset.classes

# ======================================================================
# MODEL INITIALIZATION WITH PARALLELIZATION
# ======================================================================
def initialize_model(num_classes):
    """Initialize model with CUDA and multi-GPU support"""
    # Create model with pre-trained weights
    model = timm.create_model(
        MODEL_NAME,
        pretrained=True,
        num_classes=num_classes,
        distilled=False,
        drop_rate=0.05,      # Regularization
        drop_path_rate=0.1    # Stochastic depth
    )
    
    # Enable gradient checkpointing to reduce memory usage
    model.set_grad_checkpointing()
    
    # Move model to GPU immediately
    model = model.to(device)
    
    # Enable multi-GPU training if available
    if torch.cuda.device_count() > 1:
        print(f"🚀 Using {torch.cuda.device_count()} GPUs with DataParallel!")
        model = DataParallel(model)
    
    return model

# ======================================================================
# CUDA-OPTIMIZED TRAINING LOOP
# ======================================================================
def train_model(model, train_loader, val_loader, num_epochs):
    """Training loop optimized for CUDA with parallel processing"""
    # Initialize mixed precision scaler
    scaler = torch.cuda.amp.GradScaler(enabled=device.type == 'cuda')
    
    # Loss function with label smoothing
    criterion = LabelSmoothingCrossEntropy(smoothing=0.1)
    
    # Optimizer with weight decay
    optimizer = AdamP(model.parameters(), lr=LR, weight_decay=1e-4)
    
    # Learning rate scheduling
    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer, 
        T_max=len(train_loader) * (num_epochs - WARMUP_EPOCHS),
        eta_min=LR/100
    )
    
    # Warmup scheduler
    warmup_scheduler = optim.lr_scheduler.LinearLR(
        optimizer, 
        start_factor=0.01,
        end_factor=1.0,
        total_iters=len(train_loader) * WARMUP_EPOCHS
    )
    
    # Early stopping setup
    best_val_acc = 0.0
    epochs_no_improve = 0
    early_stop = False
    history = {'train_loss': [], 'val_acc': [], 'lr': []}
    
    # Epoch loop
    for epoch in range(num_epochs):
        if early_stop:
            print(f"⏹ Early stopping triggered at epoch {epoch+1}")
            break
            
        # ---------------------
        # TRAINING PHASE
        # ---------------------
        model.train()
        train_loss = 0.0
        
        # Initialize Mixup each epoch
        mixup_fn = Mixup(
            mixup_alpha=0.2, 
            cutmix_alpha=1.0, 
            prob=0.5,
            label_smoothing=0.1,
            num_classes=NUM_CLASSES
        )
        
        # Batch processing with CUDA optimizations
        for inputs, targets in tqdm(train_loader, desc=f'Train Epoch {epoch+1}/{num_epochs}'):
            # Asynchronous data transfer to GPU (non-blocking)
            inputs = inputs.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)
            
            # Apply Mixup/Cutmix
            inputs, targets = mixup_fn(inputs, targets)
            
            optimizer.zero_grad()
            
            # Mixed precision context (FP16 operations)
            with torch.cuda.amp.autocast(enabled=device.type == 'cuda'):
                outputs = model(inputs)
                loss = criterion(outputs, targets)
            
            # Backpropagation with gradient scaling
            scaler.scale(loss).backward()
            
            # Gradient clipping to prevent explosions
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            # Optimizer step with scaling
            scaler.step(optimizer)
            scaler.update()
            
            # Update learning rate
            if epoch < WARMUP_EPOCHS:
                warmup_scheduler.step()
            else:
                scheduler.step()
            
            train_loss += loss.item()
        
        # ---------------------
        # VALIDATION PHASE
        # ---------------------
        model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, targets in tqdm(val_loader, desc='Validating'):
                inputs = inputs.to(device, non_blocking=True)
                targets = targets.to(device, non_blocking=True)
                
                # Mixed precision validation (faster)
                with torch.cuda.amp.autocast(enabled=device.type == 'cuda'):
                    outputs = model(inputs)
                
                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
        
        # Calculate metrics
        avg_loss = train_loss / len(train_loader)
        val_acc = 100 * correct / total
        
        # Update history
        history['train_loss'].append(avg_loss)
        history['val_acc'].append(val_acc)
        history['lr'].append(optimizer.param_groups[0]['lr'])
        
        print(f"Epoch {epoch+1:03d}/{num_epochs} | "
              f"Loss: {avg_loss:.4f} | "
              f"Val Acc: {val_acc:.2f}% | "
              f"LR: {optimizer.param_groups[0]['lr']:.2e}")
        
        # ---------------------
        # EARLY STOPPING LOGIC
        # ---------------------
        if val_acc > best_val_acc + MIN_DELTA:
            print(f"📈 Accuracy improved: {best_val_acc:.2f}% → {val_acc:.2f}%")
            best_val_acc = val_acc
            epochs_no_improve = 0
            
            # Save best model (handle parallel models)
            model_state = model.module.state_dict() if isinstance(model, DataParallel) else model.state_dict()
            torch.save({
                'epoch': epoch,
                'model_state_dict': model_state,
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
            }, MODEL_SAVE_PATH)
        else:
            epochs_no_improve += 1
            print(f"⏳ No improvement: {epochs_no_improve}/{PATIENCE}")
            
            if epochs_no_improve >= PATIENCE:
                early_stop = True
                print(f"🛑 Early stopping! Best accuracy: {best_val_acc:.2f}%")
    
    # Load best model for return
    checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
    if isinstance(model, DataParallel):
        model.module.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"🏁 Training complete. Best accuracy: {best_val_acc:.2f}%")
    return model, history

# ======================================================================
# VISUALIZATION
# ======================================================================
def plot_training_history(history):
    """Visualize training progress and metrics"""
    plt.figure(figsize=(15, 10))
    
    # Loss plot
    plt.subplot(2, 1, 1)
    plt.plot(history['train_loss'], label='Training Loss')
    plt.title('Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Accuracy plot
    plt.subplot(2, 1, 2)
    plt.plot(history['val_acc'], label='Validation Accuracy', color='orange')
    plt.title('Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.axhline(y=max(history['val_acc']), color='r', linestyle='--', 
                label=f'Best: {max(history["val_acc"]):.2f}%')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.show()

# ======================================================================
# MAIN EXECUTION FLOW
# ======================================================================
if __name__ == "__main__":
    print("🚀 Starting emotion classification training")
    print(f"🔧 Configuration: {MODEL_NAME} | Batch: {BATCH_SIZE} | LR: {LR}")
    
    # Prepare data
    train_loader, val_loader, class_names = prepare_data()
    print(f"📊 Classes: {class_names}")
    print(f"📦 Train batches: {len(train_loader)} | Val batches: {len(val_loader)}")
    
    # Initialize model
    model = initialize_model(NUM_CLASSES)
    param_count = sum(p.numel() for p in model.parameters())
    print(f"🧠 Model: {MODEL_NAME} | Parameters: {param_count/1e6:.1f}M")
    
    # Train model
    trained_model, history = train_model(model, train_loader, val_loader, EPOCHS)
    
    # Save final model
    final_state = trained_model.module.state_dict() if isinstance(trained_model, DataParallel) else trained_model.state_dict()
    torch.save(final_state, 'emotion_model_final.pth')
    
    # Save for inference
    trained_model.eval()
    example = torch.rand(1, 3, IMG_SIZE, IMG_SIZE).to(device)
    
    # Optimized model export
    if device.type == 'cuda':
        with torch.cuda.amp.autocast():
            traced_model = torch.jit.trace(trained_model, example)
    else:
        traced_model = torch.jit.trace(trained_model, example)
        
    traced_model.save('emotion_model_traced.pt')
    model_size = os.path.getsize('emotion_model_traced.pt') / 1e6
    print(f"💾 Model saved: {model_size:.1f}MB")
    
    # Visualize training
    plot_training_history(history)
    print("✅ Training completed successfully!")